In [1]:
!unzip -q "/content/drive/MyDrive/CV_project/data/안구질환/train_test_val.zip"


In [2]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [3]:
!find /content/train_test_val/ -name ".ipynb_checkpoints" -type d -exec rm -rf {} +


In [4]:
# 하이퍼파라미터 설정
hyperparams = {
    "batch_size": 4,  # 배치 크기: 한 번에 네 개의 이미지를 처리
    "learning_rate": 0.0001,  # 학습률: 모델이 학습할 때 가중치를 업데이트하는 속도
    "epochs": 10,  # 에포크 수: 전체 데이터셋을 몇 번 반복해서 학습할 것인지 설정
    "transform": transforms.Compose(  # 데이터 변환 파이프라인 설정
        [
            transforms.Resize(256),  # 이미지를 256x256 크기로 리사이즈
            transforms.CenterCrop(224),  # 이미지를 중앙에서 224x224 크기로 자름
            transforms.ToTensor(),  # 이미지를 텐서로 변환 (픽셀 값을 [0, 1] 범위로 변환)
            transforms.Normalize(mean=[0.48235, 0.45882, 0.40784], std=[1.0/255.0, 1.0/255.0, 1.0/255.0])
            # 이미지 정규화: 평균과 표준편차를 사용하여 각 채널을 정규화
        ]
    )
}



### pytorch 이미지 폴더, 이미지 로더 설정

In [5]:
# 학습 데이터셋 생성
train_dataset = ImageFolder(root="/content/train_test_val/train", transform=hyperparams["transform"])

# 테스트 데이터셋 생성
test_dataset = ImageFolder(root="/content/train_test_val/test", transform=hyperparams["transform"])

# 학습 데이터 로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True)

# 테스트 데이터 로더 생성
test_dataloader = DataLoader(test_dataset, batch_size=hyperparams["batch_size"], shuffle=True, drop_last=True)


### 모델 불러오기


In [6]:
model = models.vgg16(weights="VGG16_Weights.IMAGENET1K_V1")
model.classifier[6] = nn.Linear(4096, len(train_dataset.classes))

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 222MB/s]


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=hyperparams["learning_rate"])

In [ ]:
for epoch in range(hyperparams["epochs"]):
    cost = 0.0

    for images, classes in train_dataloader:
        images = images.to(device)
        classes = classes.to(device)

        output = model(images)
        loss = criterion(output, classes)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 1.315
Epoch :    2, Cost : 1.102
Epoch :    3, Cost : 1.022
Epoch :    4, Cost : 0.963
Epoch :    5, Cost : 0.912
Epoch :    6, Cost : 0.865
Epoch :    7, Cost : 0.829
Epoch :    8, Cost : 0.795


In [ ]:
with torch.no_grad():
    model.eval()

    accuracy = 0.0
    for images, classes in test_dataloader:
        images = images.to(device)
        classes = classes.to(device)

        outputs = model(images)
        probs = F.softmax(outputs, dim=-1)
        outputs_classes = torch.argmax(probs, dim=-1)

        accuracy += int(torch.eq(classes, outputs_classes).sum())

    print(f"acc@1 : {accuracy / (len(test_dataloader) * hyperparams['batch_size']) * 100:.2f}%")

In [ ]:
torch.save(model.state_dict(), "../models/VGG16.pt")
print("Saved the model weights")